In [1]:
from pyspark import SparkContext
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis.constants import SPATIAL, ZOOM, TILE
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.rdd import RasterRDD, TiledRasterRDD
from geopyspark.geotrellis.catalog import _construct_catalog, _mapped_cached, read_layer_metadata
from geopyspark.geotrellis.constants import SPATIAL, TILE
from geonotebook.vis.geotrellis.render_methods import render_nlcd, single_band_render_from_color_map
from geonotebook.wrappers import GeoTrellisCatalogLayerData, RddRasterData
from geonotebook.wrappers.vector import GeoJsonData
import numpy as np
import re
from shapely.geometry import Polygon, mapping
from shapely.ops import transform
from functools import partial, reduce
import pyproj
import rasterio

In [2]:
catalog_uri = "s3://otid-data/input/gt-processed-dem/"

In [3]:
full_min = -35.68600001263555
full_max = 106.0929250034017
    

In [4]:
def bounds_to_shape(bounds):
    return Polygon([(bounds.left, bounds.top), 
                    (bounds.right, bounds.top),
                    (bounds.right, bounds.bottom),
                    (bounds.left, bounds.bottom)])

def extent_to_shape(extent):
    return Polygon([(extent['xmin'], extent['ymax']), 
                    (extent['xmax'], extent['ymax']),
                    (extent['xmax'], extent['ymin']),
                    (extent['xmin'], extent['ymin'])])

def reprojected(shape):
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:32633'),
        pyproj.Proj(init='epsg:4326'))
    return transform(project, shape)


In [5]:
sc = SparkContext(appName="Potsdam")
geopysc = GeoPyContext(sc)

In [6]:
from boto3 import client
bucket = "otid-data"
keys = []
conn = client('s3')
for key in conn.list_objects(Bucket=bucket, Prefix='input/1_DSM_normalisation_geotiff-with-geo/')['Contents']:
    keys.append(key['Key'])


In [7]:
## A bit of a hack, find the min max of the rasters
## we are dealing with before saving off, since we need to 
## rescale and convert
def find_min_max(key):
    m = re.search(r"\_(\d\d)\_(\d\d)", key)
    k = m.group(0)
    src = rasterio.open('s3://otid-data/%s' % key)
    try:
        bounds = bounds_to_shape(src.bounds)
        ll_bounds = reprojected(bounds)
        M.set_center(ll_bounds.centroid.x, ll_bounds.centroid.y, 15)
        vd = GeoJsonData(mapping(ll_bounds))
        name = "Tile%s" % k
        print("Processing %s with center %s" % (name, ll_bounds.centroid))
        print("  Bounds: %s" % (bounds))
        l = M.add_layer(vd, name=name, colors=[0x00FF00])
        try:
            dem_all = query(geopysc, SPATIAL, catalog_uri, "potsdam-dsm-all", 0, intersects=bounds)
            dem_ground = query(geopysc, SPATIAL, catalog_uri, "potsdam-dsm-ground", 0, intersects=bounds)
            dem = dem_all.repartition(100) - dem_ground.repartition(100)
            mm = dem.get_min_max()
            M.remove_layer(M.layers.find(name))
            M.add_layer(vd, name=name, colors=[0x0000FF])
            return mm
        except:
            M.remove_layer(M.layers.find(name))
            M.add_layer(vd, name=name, colors=[0xFF0000])
    finally:
        src.close()        


In [8]:
#mms = list(map(find_min_max, keys))
#total_min = 10000.0
#total_max = -10000.0
#for m, M in mms:
#    if m < total_min:
#        total_min = m
#    if total_max < M:
#        total_max = M
#print((total_min, total_max))
total_min, total_max = (-48.582757268436076, 58.07267200631878)

In [19]:
def process_tile(key):
    m = re.search(r"\_(\d\d)\_(\d\d)", key)
    k = m.group(0)
    src = rasterio.open('s3://otid-data/%s' % key)
    try:
        bounds = bounds_to_shape(src.bounds)
        ll_bounds = reprojected(bounds)
        M.set_center(ll_bounds.centroid.x, ll_bounds.centroid.y, 15)
        vd = GeoJsonData(mapping(ll_bounds))
        name = "Tile%s" % k
        print("Processing %s with center %s" % (name, ll_bounds.centroid))
        print("  Bounds: %s" % (bounds))
        l = M.add_layer(vd, name=name, colors=[0x00FF00])
        try:
            dem = query(geopysc, SPATIAL, catalog_uri, "potsdam-dsm-full", 0, intersects=bounds)
            dem.repartition(100) \
               .save_stitched("/home/hadoop/notebooks/dsm_potsdam%s.tif" % k, 
                              crop_bounds=bounds.bounds, 
                              crop_dimensions=[6000,6000])
            M.remove_layer(M.layers.find(name))
            M.add_layer(vd, name=name, colors=[0x0000FF])
        except:
            M.remove_layer(M.layers.find(name))
            M.add_layer(vd, name=name, colors=[0xFF0000])
    finally:
        src.close()

def process_tile_with_norm(key):
    m = re.search(r"\_(\d\d)\_(\d\d)", key)
    k = m.group(0)
    src = rasterio.open('s3://otid-data/%s' % key)
    try:
        bounds = bounds_to_shape(src.bounds)
        ll_bounds = reprojected(bounds)
        M.set_center(ll_bounds.centroid.x, ll_bounds.centroid.y, 15)
        vd = GeoJsonData(mapping(ll_bounds))
        name = "Tile%s" % k
        print("Processing %s with center %s" % (name, ll_bounds.centroid))
        print("  Bounds: %s" % (bounds))
        l = M.add_layer(vd, name=name, colors=[0x00FF00])
        #try:
        dem_all = query(geopysc, SPATIAL, catalog_uri, "potsdam-dsm-all", 0, intersects=bounds)
        dem_ground = query(geopysc, SPATIAL, catalog_uri, "potsdam-dsm-ground", 0, intersects=bounds)
        dem = dem_all.repartition(100) - dem_ground.repartition(100)
        dem = dem.normalize(total_min, total_max, 0.0, 256.0)
        dem = dem.convert_data_type("uint8")
        print("MIN MAX: (%d, %d)" % dem.get_min_max())
        dem.save_stitched("/home/hadoop/notebooks/dsm_potsdam%s.tif" % k, 
                              crop_bounds=bounds.bounds, 
                              crop_dimensions=[6000,6000])
        M.remove_layer(M.layers.find(name))
        M.add_layer(vd, name=name, colors=[0x0000FF])
        #except:
        #    M.remove_layer(M.layers.find(name))
        #    M.add_layer(vd, name=name, colors=[0xFF0000])
    finally:
        src.close()

In [20]:
process_tile_with_norm(keys[0])

Processing Tile_02_10 with center POINT (13.04651140975037 52.40978631048631)
  Bounds: POLYGON ((366976.5 5808562.6, 367276.5 5808562.6, 367276.5 5808262.6, 366976.5 5808262.6, 366976.5 5808562.6))
MIN MAX: (95, 185)


In [21]:
for key in keys:
    # process_tile(key)
    process_tile_with_norm(key)


Processing Tile_02_10 with center POINT (13.04651140975037 52.40978631048631)
  Bounds: POLYGON ((366976.5 5808562.6, 367276.5 5808562.6, 367276.5 5808262.6, 366976.5 5808262.6, 366976.5 5808562.6))
MIN MAX: (95, 185)
Processing Tile_02_11 with center POINT (13.05091919593039 52.40985908050819)
  Bounds: POLYGON ((367276.5 5808562.6, 367576.5 5808562.6, 367576.5 5808262.6, 367276.5 5808262.6, 367276.5 5808562.6))
MIN MAX: (105, 181)
Processing Tile_02_12 with center POINT (13.05532700090658 52.40993168628162)
  Bounds: POLYGON ((367576.5 5808562.6, 367876.5 5808562.6, 367876.5 5808262.6, 367576.5 5808262.6, 367576.5 5808562.6))
MIN MAX: (96, 164)
Processing Tile_02_13 with center POINT (13.05973482463657 52.41000412780495)
  Bounds: POLYGON ((367876.5 5808562.6, 368176.5 5808562.6, 368176.5 5808262.6, 367876.5 5808262.6, 367876.5 5808562.6))
MIN MAX: (83, 195)
Processing Tile_02_14 with center POINT (13.06414266707799 52.41007640507662)
  Bounds: POLYGON ((368176.5 5808562.6, 368476.5 

MIN MAX: (97, 221)


### Get the extent of the data

In [6]:
def get_bounds(key):
    m = re.search(r"\_(\d\d)\_(\d\d)", key)
    k = m.group(0)
    src = rasterio.open('s3://otid-data/%s' % key)
    try:
        return bounds_to_shape(src.bounds)
    finally:
        src.close()

bounds = []
for key in keys:
    bounds.append(get_bounds(key))


In [12]:
p = reduce(lambda a, b: a.union(b), bounds)


In [15]:
p.envelope.bounds


(366076.5, 5806762.6, 368776.5, 5808562.6)

### Debug code

In [23]:
## Figure out the bounds of the DEM data in Web Mercator
import json
lmd = read_layer_metadata(geopysc, SPATIAL, catalog_uri, "potsdam", 0)
e = lmd['extent']
s = extent_to_shape(e)

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:32633'),
    pyproj.Proj(init='epsg:3857'))
rs = transform(project, s)
json.dumps(mapping(rs))

'{"type": "Polygon", "coordinates": [[[1450606.9863003485, 6874761.1777096335], [1453551.0983858046, 6874841.041727237], [1453630.1872897947, 6871890.04526937], [1450687.1503113946, 6871810.256835365], [1450606.9863003485, 6874761.1777096335]]]}'

In [ ]:
# There was some data missing, debug code to figure that out
key = keys[3]
src = rasterio.open('s3://otid-data/%s' % key)
bounds = bounds_to_shape(src.bounds)

In [ ]:
list(bounds.bounds)

In [ ]:
process_tile(keys[3])

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
d = GeoJsonData(mapping(ll_bounds))

In [ ]:
M.add_layer(d, name="test", colors=[0xFFFF00])

In [ ]:
ll_bounds = reprojected(bounds)
M.set_center(ll_bounds.centroid.x, ll_bounds.centroid.y, 15)

In [ ]:
bounds.centroid.x


In [9]:
len(set(keys))

38

In [10]:
keys

['input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_02_10_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_02_11_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_02_12_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_02_13_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_02_14_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_03_10_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_03_11_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_03_12_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_03_13_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_03_14_normalized_lastools-geo.tif',
 'input/1_DSM_normalisation_geotiff-with-geo/dsm_potsdam_04_10_normali